In [ ]:
import os
import numpy as np
import h5py
import pandas as pd
import pickle
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
plt.rcParams['text.latex.unicode'] = False
import matplotlib
# important for text to be detecting when importing saved figures into illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams["font.family"] = "Arial"

In [ ]:
# important arguments; USER DEFINED VARIABLES
raw_npilCorr = 0 # 0 for raw vs 1 neuropil corrected signal
fs = 6

rois_to_include =  np.arange(5) #[0,2,3,6] # [0,1,20,21,26] #

fname_signals = 'VJ_OFCVTA_7_260_D6_neuropil_corrected_signals_15_50_beta_0.8.npy'
fdir = os.path.abspath('./sample_data') # directory where suite2p folder is
filename = os.path.split(fdir)[-1]

proj_img_path = os.path.join(fdir, f'{filename}_output_images\std_img.tif')
sima_h5_path = os.path.join(fdir, f'{filename}_sima_mc.h5')
sima_mask_path = os.path.join(fdir, f'{filename}_sima_masks.npy')

fig_save_dir = os.path.join(fdir, 'figs')
if not os.path.exists(fig_save_dir):
    os.mkdir(fig_save_dir)



# SIMA 

In [ ]:
roi_signal_sima = np.load(os.path.join(fdir, fname_signals))

In [ ]:
if os.path.exists(proj_img_path):
    proj_manual = {}
    proj_manual['std_img'] = np.array(Image.open(proj_img_path))

    manual_data_dims = proj_manual['std_img'].shape
elif os.path.exists(sima_h5_path):
    # load video data
    # open h5 to read, find data key, grab data, then close
    h5 = h5py.File(sima_h5_path,'r')
    sima_data = np.squeeze(np.array( h5[list(h5.keys())[0]] )).astype('int16') # np.array loads all data into memory
    h5.close()

    proj_manual = {'mean_img': np.mean(sima_data, axis = 0), 
                   'max_img': np.max(sima_data, axis = 0), 
                   'std_img': np.std(sima_data, axis = 0) }

    manual_data_dims = sima_data.shape

In [ ]:
# grab ROI masks from sima (these are probably manually drawn ROIs from imagej)
sima_masks = np.load(sima_mask_path)
numROI_sima = sima_masks.shape[0]

In [ ]:
# define number of ROIs to visualize and make colormap for contours
colors_roi = plt.cm.viridis(np.linspace(0,1,len(rois_to_include)))

In [ ]:
# make binary array of roi pixels for contour plotting
zero_template_manual = np.zeros([manual_data_dims[-2], manual_data_dims[-1]])
roi_label_loc_manual = []

for idx, iROI in enumerate(rois_to_include):
    
    # make binary map of ROI pixels
    ypix_roi, xpix_roi = np.where(sima_masks[iROI,:,:] == 1)
    
    if ypix_roi.size == 0:

        roi_label_loc_manual.append( [0, 0] )
        
    else:
        zero_template_manual[ ypix_roi, xpix_roi ] = idx+1

        roi_label_loc_manual.append( [np.min(ypix_roi), np.min(xpix_roi)] )


In [ ]:
# plot contours and cell numbers
clims = [ np.min(proj_manual['std_img']), 
        np.max(proj_manual['std_img']) ]

fig, ax = plt.subplots(1, 1, figsize = (10,10))
im = ax.imshow(proj_manual['std_img'], cmap = 'gray', vmin = clims[0]*1, vmax = clims[1]*0.8)
ax.axis('off')

for iROI, roi_id in enumerate(rois_to_include): 
    ax.contour(sima_masks[roi_id,:,:], colors=[colors_roi[iROI]])

# plot ROI number
for iROI, roi_id in enumerate(rois_to_include): 
    ax.text(roi_label_loc_manual[iROI][1]-6, roi_label_loc_manual[iROI][0]-6,  str(roi_id), fontsize=15, color = colors_roi[iROI]);

plt.title('Manual ROIs', fontsize = 20)
#plt.axis([150,400,20,270])
#plt.gca().invert_yaxis()

#fig.colorbar(im)
#im.set_clim([30, 350])

plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.jpg'))
plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.pdf'))

In [ ]:
# initialize variables for plotting time-series

num_samps = roi_signal_sima.shape[-1]
total_time = num_samps/fs 
tvec = np.linspace(0,total_time,num_samps)

In [ ]:
fig, ax = plt.subplots(len(rois_to_include), 1, figsize = (7,30), sharey=True)
for counter, iROI in enumerate(rois_to_include):
    
    baseline = np.mean(roi_signal_sima[iROI,:])
    
    to_plot = ( (roi_signal_sima[iROI,:]-baseline)/baseline )*100
    
    ax[counter].plot(tvec, np.transpose( to_plot ), color = colors_roi[counter], linewidth=0.8);

    if counter == 0:
        ax[counter].set_ylabel('dF/F',fontsize = 20)
        
    ax[counter].autoscale(enable=True, axis='x', tight=True) 
    ax[counter].tick_params(axis='both', which='major', labelsize=15)
    ax[counter].tick_params(axis='both', which='minor', labelsize=15)
    #ax[counter].set_xlim([300,2000])
    
ax[counter].set_xlabel('Time (s)',fontsize = 20);

plt.savefig(os.path.join(fig_save_dir, 'roi_ts.jpg'))
plt.savefig(os.path.join(fig_save_dir, 'roi_ts.pdf'))

